In [ ]:
from fastai.vision.all import *

import pandas as pd

In [ ]:
data_path = Path('/home/tanul/torchenv/data/tiles') #todo: move this out to /4TB
data_path.ls()

In [ ]:
LABELS_FILE = '/home/gosia/text-detection-internships/FasterRCNN Pipeline/train.csv'
labels_df = pd.read_csv(LABELS_FILE)
labels_df

In [ ]:
labels_df.label.unique()

In [ ]:
labels_df.loc[labels_df['label'] == ']']

In [ ]:
def df_to_dict(df):
    out = {}
    for idx, row in df.iterrows():
        fname = row['image_id']
        annot = out.get(fname, ([], []))
        annot[0].append([row['xmin'], row['xmax'], row['ymin'], row['ymax']])
        annot[1].append(row['label'])
        out[fname] = (annot)
    return out

In [ ]:
img2bbox = df_to_dict(labels_df)
img2bbox

In [ ]:
dblocks = DataBlock(blocks=(ImageBlock, BBoxBlock, BBoxLblBlock),
                    splitter=RandomSplitter(),
                    get_items=get_image_files,
                    get_y=[lambda o: img2bbox[o.name][0], lambda o: img2bbox[o.name][1]],
                    n_inp=1
                   )

In [ ]:
dls = dblocks.dataloaders(data_path)

In [ ]:
dblocks.summary(data_path)

In [ ]:
dls.show_batch()